# Eigenvector Centrality

In this notebook, we will compute the Eigenvector centrality for vertice in our test database using cuGraph and NetworkX. The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware |
| --------------|------------|------------------|-----------------|----------------|
| Don Acosta    | 07/05/2022 | created          | 22.08 nightly   | DGX Tesla V100 CUDA 11.5

## Introduction
Eigenvector centrality computes the centrality for a vertex based on the
centrality of its neighbors. The Eigenvector of a node measures influence within a graph by taking into account a vertex's connections to other highly connected vertices.


See [Eigenvector Centrality on Wikipedia](https://en.wikipedia.org/wiki/Eigenvector_centrality) for more details on the algorithm.



The eigenvector centrality for node i is the
i-th element of the vector x defined by the eigenvector equation.

<img src="https://latex.codecogs.com/png.latex?c_B(v)&space;=\sum_{s,t&space;\in&space;V}&space;\frac{\sigma(s,&space;t|v)}{\sigma(s,&space;t)}" title="c_B(v) =\sum_{s,t \in V} \frac{\sigma(s, t|v)}{\sigma(s, t)}" />


To compute the Eigenvector centrality scores for a graph in cuGraph we use:<br>


    G : cuGraph.Graph or networkx.Graph
    max_iter : int, optional (default=100)
        The maximum number of iterations before an answer is returned. This can
        be used to limit the execution time and do an early exit before the
        solver reaches the convergence tolerance.
    tol : float, optional (default=1e-6)
        Set the tolerance the approximation, this parameter should be a small
        magnitude value.
        The lower the tolerance the better the approximation. If this value is
        0.0f, cuGraph will use the default value which is 1.0e-6.
        Setting too small a tolerance can lead to non-convergence due to
        numerical roundoff. Usually values between 1e-2 and 1e-6 are
        acceptable.
    normalized : bool, optional, default=True
        If True normalize the resulting eigenvector centrality values

   

Returns:

    df : cudf.DataFrame or Dictionary if using NetworkX
        GPU data frame containing two cudf.Series of size V: the vertex
        identifiers and the corresponding eigenvector centrality values.
        df['vertex'] : cudf.Series
            Contains the vertex identifiers
        df['eigenvector_centrality'] : cudf.Series
            Contains the eigenvector centrality of vertices

### _NOTICE_
cuGraph does not currently support the ‘endpoints’ and ‘weight’ parameters as seen in the corresponding networkX call. 

#### Some notes about vertex IDs...
* The current version of cuGraph requires that vertex IDs be representable as 32-bit integers, meaning graphs currently can contain at most 2^32 unique vertex IDs. However, this limitation is being actively addressed and a version of cuGraph that accommodates more than 2^32 vertices will be available in the near future.
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../../img/zachary_black_lines.png)


Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.


### Prep

In [1]:
# Import needed libraries
import cugraph
import cudf

In [2]:
# NetworkX libraries
import networkx as nx

### Some Prep

In [3]:
# Define the path to the test data  
datafile='../../data/karate-data.csv'

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [4]:
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

### Create a Graph 

In [5]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph(directed=True)
G.from_cudf_edgelist(gdf, source='src', destination='dst')

### Call the Eigenvector Centrality algorithm

In [6]:
# Call cugraph.eigenvector_centrality 
k_df = cugraph.eigenvector_centrality(G, max_iter=1000)
k_df = k_df.sort_values("vertex").reset_index(drop=True)
k_df


,eigenvector_centrality,vertex
0,0.355491,1
1,0.265960,2
2,0.317192,3
3,0.211180,4
4,0.075969,5
5,0.079483,6
6,0.079483,7
7,0.170960,8
8,0.227404,9
9,0.102675,10


_It was that easy!_  

----

Let's now look at the results

---

## Now compute using NetworkX

In [7]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

As mentioned, the scores are different but the ranking is the same.

___
Copyright (c) 2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___